In [ ]:
#r "nuget:Microsoft.ML"

Installed Packages Microsoft.ML, 1.6.0

In [ ]:
using System;
using System.IO;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var trainDataPath = "../datasets//weatherAUS-train.csv";
var testDataPath = "../datasets//weatherAUS-test.csv";

In [ ]:
public class WeatherData
{   
    [LoadColumn(0)]
    public float MinTemp;
    
    [LoadColumn(1)]
    public float MaxTemp;
    
    [LoadColumn(2)]
    public float Rainfall;
    
    [LoadColumn(3)]
    public float WindGustSpeed;
    
    [LoadColumn(4)]
    public float WindSpeed9am;
    
    [LoadColumn(5)]
    public float WindSpeed3pm;
    
    [LoadColumn(6)]
    public float Humidity9am;
    
    [LoadColumn(7)]
    public float Humidity3pm;
    
    [LoadColumn(8)]
    public float Pressure9am;
    
    [LoadColumn(9)]
    public float Pressure3pm;
    
    [LoadColumn(10)]
    public float Temp9am;
    
    [LoadColumn(11)]
    public float Temp3pm;
    
    [LoadColumn(12)]
    public float RainToday;
    
    [LoadColumn(13)]
    public bool RainTomorrow;
}

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
var trainingDataView = mlContext.Data.LoadFromTextFile<WeatherData>(trainDataPath, hasHeader: true, separatorChar: ',');
var testDataView = mlContext.Data.LoadFromTextFile<WeatherData>(testDataPath, hasHeader: true, separatorChar: ',');

In [ ]:
var dataProcessPipeline = mlContext.Transforms.Conversion.MapValueToKey(outputColumnName: "KeyColumn", inputColumnName: nameof(WeatherData.RainTomorrow))
                        .Append(mlContext.Transforms.Concatenate("Features", nameof(WeatherData.MinTemp),
                                                                              nameof(WeatherData.MaxTemp),
                                                                              nameof(WeatherData.Rainfall),
                                                                              nameof(WeatherData.WindGustSpeed),
                                                                              nameof(WeatherData.WindSpeed9am),
                                                                              nameof(WeatherData.WindSpeed3pm),
                                                                              nameof(WeatherData.Humidity9am),
                                                                              nameof(WeatherData.Humidity3pm),
                                                                              nameof(WeatherData.Pressure9am),
                                                                              nameof(WeatherData.Pressure3pm),
                                                                              nameof(WeatherData.Temp9am),
                                                                              nameof(WeatherData.Temp3pm),
                                                                              nameof(WeatherData.RainToday)))
                                                                              .AppendCacheCheckpoint(mlContext);

In [ ]:
var linearSvmTrainer = mlContext.BinaryClassification.Trainers.LinearSvm(labelColumnName: "KeyColumn", featureColumnName: "Features");
var trainer = mlContext.MulticlassClassification.Trainers.OneVersusAll(linearSvmTrainer, labelColumnName: "KeyColumn");

var trainingPipeline = dataProcessPipeline.Append(trainer);

In [ ]:
var trainedModel = trainingPipeline.Fit(trainingDataView);

In [ ]:
var predictions = trainedModel.Transform(testDataView);

In [ ]:
var metrics = mlContext.MulticlassClassification.Evaluate(data: predictions, labelColumnName: "KeyColumn", scoreColumnName: "Score");

In [ ]:
Console.WriteLine($"************************************************************");
Console.WriteLine($"*       MacroAccuracy: {metrics.MacroAccuracy}");
Console.WriteLine($"*       MicroAccuracy: {metrics.MicroAccuracy}");
Console.WriteLine($"************************************************************");

************************************************************
*       MacroAccuracy: 0,706336951469255
*       MicroAccuracy: 0,8433397441257631
************************************************************
